<a href="https://colab.research.google.com/github/harshv47/MNIST-CNN/blob/master/mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix






#from keras.datasets import mnist 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_df = pd.read_csv('/content/drive/My Drive/Dataset/mnist/train.csv')
test_df = pd.read_csv('/content/drive/My Drive/Dataset/mnist/test.csv')
print(train_df.shape)
print(test_df.shape)

(42000, 785)
(28000, 784)


In [0]:
X = train_df.drop(['label'], 1).values
Y = train_df['label'].values
test_x = test_df.values

In [0]:
# Normalize and reshape
X = X / 255.0
test_x = test_x / 255.0
X = X.reshape(-1,28,28,1)
test_x = test_x.reshape(-1,28,28,1)

In [0]:
# one hot vectors encoding 3 -> [0,0,0,1,0,0,0,0,0,0]
from keras.utils.np_utils import to_categorical


Y = to_categorical(Y)
print(Y.shape)

(42000, 10)


In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=10)

In [0]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((37800, 28, 28, 1), (4200, 28, 28, 1), (37800, 10), (4200, 10))

In [0]:
mean = np.mean(X_train)
std = np.std(X_train)

def standardize(x):
    return (x-mean)/std

In [0]:
epochs = 50
batch_size = 64

In [0]:
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Conv2D, Lambda, MaxPooling2D
from tensorflow.python.keras.layers import Dense, Dropout, Flatten 
from tensorflow.python.keras.layers.normalization import BatchNormalization


model=Sequential()

model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())    
model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))
    
model.add(MaxPooling2D(pool_size=(2,2)))
    
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512,activation="relu"))
    
model.add(Dense(10,activation="softmax"))
    
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [0]:
# Data augmentation to prevent overfitting
from keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#datagen.fit(X_train)
train_gen = datagen.flow(X_train, Y_train, batch_size=batch_size)
test_gen = datagen.flow(X_test, Y_test, batch_size=batch_size)

In [0]:
# This makes accuracy 0.998
model.fit(X, Y, batch_size=batch_size, validation_split=0.2, epochs=10)

Epoch 1/10
525/525 [==============================] - 5s 10ms/step - loss: 0.0082 - accuracy: 0.9977 - val_loss: 0.0080 - val_accuracy: 0.9977
Epoch 2/10
525/525 [==============================] - 5s 9ms/step - loss: 0.0034 - accuracy: 0.9991 - val_loss: 0.0074 - val_accuracy: 0.9986
Epoch 3/10
525/525 [==============================] - 5s 9ms/step - loss: 0.0017 - accuracy: 0.9994 - val_loss: 0.0075 - val_accuracy: 0.9981
Epoch 4/10
525/525 [==============================] - 5s 9ms/step - loss: 0.0034 - accuracy: 0.9987 - val_loss: 0.0118 - val_accuracy: 0.9973
Epoch 5/10
525/525 [==============================] - 5s 9ms/step - loss: 0.0037 - accuracy: 0.9989 - val_loss: 0.0121 - val_accuracy: 0.9974
Epoch 6/10
525/525 [==============================] - 5s 9ms/step - loss: 0.0032 - accuracy: 0.9991 - val_loss: 0.0159 - val_accuracy: 0.9962
Epoch 7/10
525/525 [==============================] - 5s 9ms/step - loss: 0.0058 - accuracy: 0.9983 - val_loss: 0.0101 - val_accuracy: 0.9973
Epoch

In [0]:
# Fit the model
history = model.fit_generator(train_gen, 
                              epochs = epochs, 
                              steps_per_epoch = X_train.shape[0] // batch_size,
                              validation_data = test_gen,
                              validation_steps = X_test.shape[0] // batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
590/590 [==============================] - 21s 35ms/step - loss: 0.1557 - accuracy: 0.9514 - val_loss: 0.1346 - val_accuracy: 0.9599
Epoch 2/50
590/590 [==============================] - 20s 34ms/step - loss: 0.0674 - accuracy: 0.9797 - val_loss: 0.0774 - val_accuracy: 0.9757
Epoch 3/50
590/590 [==============================] - 20s 33ms/step - loss: 0.0561 - accuracy: 0.9833 - val_loss: 0.0507 - val_accuracy: 0.9849
Epoch 4/50
590/590 [==============================] - 20s 33ms/step - loss: 0.0498 - accuracy: 0.9851 - val_loss: 0.0694 - val_accuracy: 0.9776
Epoch 5/50
590/590 [==============================] - 20s 33ms/step - loss: 0.0439 - accuracy: 0.9871 - val_loss: 0.0384 - val_accuracy: 0.9882
Epoch 6/50
590/590 [==============================] - 20s 33ms/step - loss: 0.0398 - accuracy: 0.9880 - val_loss: 0.0609 - val_accuracy: 0.9841
Epoch 7/50
590/590 [==============================] - 20s 33